# BN折叠
BN折叠属于是在推理上的一个小技巧，也就是折叠`Batch Normalization`。在卷积或者全连接层后面通常会有一个批量归一化操作，用来归一化数据，一般还会接上激活函数，构成了`conv+BN+ReLU`这也的基本组件。在部署推理的时候，前向推理框架一般会自动的将 BN 和它前面的卷积层折叠在一起，实现高效的前向推理网络。

下面，我们通过公式来推理这个过程：
1. **卷积层的计算公式可以表示为：**

    $ Y = W * X + B $

2. **然后 BN 层的计算公式为：**

    $\mu = \frac{1}{m}\sum_{i=1}^mx_i$

    $\sigma^2=\frac{1}{m}\sum_{i=1}^m(x_i-\mu)^2$

    $x_i=\frac{x_i-\mu}{\sqrt{\sigma^2+ \epsilon}}$

    $y_i=\gamma * x_i + \beta$

3. **将这几个公式整合到一起，得到公式如下：**

    $Y=\gamma*(\frac{(W*X+B)-\mu}{\sqrt{\sigma^2+\epsilon}})+\beta$

    $Y=\frac{\gamma*W}{\sqrt{\sigma^2+\epsilon}}*X+\frac{\gamma*(B-\mu)}{\sqrt{\sigma^2+\epsilon}}+\beta$

4. 得到合并后的权重和偏置
    然后令$a = \frac{\gamma}{\sqrt{\delta^2+\epsilon}}$

    **那么，合并 BN 层后的卷积层的权重和偏置可以表示为：**

    $W_{merged}=W*a$

    $B_{merged}=(B-\mu)*a+\beta$

    **值得一提的是，一般 Conv 后面接 BN 的时候很多情况下是不带 Bias 的，这个时候上面的公式就会少第二项。**


In [ ]:
# 代码示例。来源：http://giantpandacv.com/project/%E9%83%A8%E7%BD%B2%E4%BC%98%E5%8C%96/AI%20%E9%83%A8%E7%BD%B2%E5%8F%8A%E5%85%B6%E5%AE%83%E4%BC%98%E5%8C%96%E7%AE%97%E6%B3%95/%E6%8A%98%E5%8F%A0BN%E5%B1%82/
def fuse(conv, bn):
    global i
    i = i + 1
    # ********************BN参数*********************
    mean = bn.running_mean
    var_sqrt = torch.sqrt(bn.running_var + bn.eps)
    gamma = bn.weight
    beta = bn.bias
    # *******************conv参数********************
    w = conv.weight
    if conv.bias is not None:
        b = conv.bias
    else:
        b = mean.new_zeros(mean.shape)

    if(i >= 2 and i <= 7):
        b = b - mean + beta * var_sqrt
    else:
        w = w * (gamma / var_sqrt).reshape([conv.out_channels, 1, 1, 1])
        b = (b - mean)/var_sqrt * gamma + beta
    fused_conv = nn.Conv2d(conv.in_channels,
                        conv.out_channels,
                        conv.kernel_size,
                        conv.stride,
                        conv.padding,
                        groups=conv.groups,
                        bias=True)
    fused_conv.weight = nn.Parameter(w)
    fused_conv.bias = nn.Parameter(b)
    return fused_conv

还有一个将`Mo Mobilenet Caffe` 模型的卷积和 BN 进行合并的代码，感兴趣的自行尝试：[https://github.com/BBuf/cv_tools/blob/master/merge_bn.py](https://github.com/BBuf/cv_tools/blob/master/merge_bn.py)

## 在量化感知训练中的应用
因为，在推理时，会自动进行BN折叠，训练后量化也是对BN折叠后的`Weights`和`bias`进行量化，而以往在正常训练模型时，是不进行BN折叠的，`Batch Normalization`作为单独的算子来进行计算。如果在量化感知训练的过程中，不进行BN折叠，就会导致一个问题：

**以conv+BN为例， QAT过程中，会分别对conv和BN的参数进行伪量化，这就与推理阶段的折叠后量化不一致。 为了能够在QAT过程中，更好的模拟推理阶段的量化效果，就需要也模拟这种折叠过程。**

QAT 中常见的算子折叠组合还有：Conv + BN、Conv + BN + ReLU、Conv + ReLU、Linear + ReLU、BN + ReLU